# BD Agent Notebook

Notebook for BD Agent. Let's do this starting from search & summary node as tool to call, BD agent to decide what tools to use and advisor chain

## Prepare data for BD Agents (Schema Context, Business Profile)

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import glob

import sys

from dotenv import load_dotenv
load_dotenv()

_path = os.getcwd()
while "graphs" not in os.listdir(_path):

	_path = os.path.dirname(_path)

	if _path.replace("/", "").strip() == "":

		raise FileNotFoundError()
 
	if _path not in sys.path:

		sys.path.append(_path)

		print(f"Appending {_path} to path")

from graphs.answer_graph import build_answer_graph
from graphs.understand_graph import build_understand_graph  # if needed in later phases
from graphs.state import AgentState

import nest_asyncio
nest_asyncio.apply()

from utils.scraper import scrape_urls
from utils.sanitize import sanitize

from logger_config import setup_logger
logger = setup_logger()
logger.info("StratPilot started.")

Appending /mnt/d/Agentic Hackathon/agentic-app/workspace to path
Appending /mnt/d/Agentic Hackathon/agentic-app to path


2025-04-27 10:15:59,956 - stratpilot - INFO - StratPilot started.


In [2]:
#Prepare data (Read)

uploaded_files = glob.glob("/mnt/d/Agentic Hackathon/agentic-app/data/coffee-shop-sample-data/*")
datasets = {}
care = ["customer", "201904 sales reciepts", "product"]

for file in uploaded_files:
    if not any(c in file for c in care):
        continue

    df = pd.read_csv(file)
    dataset_label = os.path.basename(file).split(".")[0]

    inferred = {col: col for col in df.columns}
    column_descriptions = {}
    for col, guess in inferred.items():
        column_descriptions[col] = f"This column likely represents {guess}."

    datasets[sanitize(dataset_label)] = {
        "data": df,
        "description": dataset_label,
        "column_descriptions": column_descriptions
    }

all_dataframes = {}
all_column_schemas = {}

for dataset_name, bundle in datasets.items():
    description = bundle.get("description", "").strip()
    df = bundle["data"]
    column_descriptions = bundle.get("column_descriptions", {})

    all_dataframes[dataset_name] = df
    all_column_schemas[dataset_name] = column_descriptions

business_profile = {
  'type': 'Cafe',
  'details': 'Independent neighborhood cafe located in a busy urban area, serving specialty coffee and light meals. Targets young professionals and students. Medium foot traffic, strong competition from chain coffee shops. Focused on in-store experience and Instagram-friendly ambiance. Operates 7 days a week with peak hours in the morning and late afternoon. Interested in increasing repeat customers and promoting seasonal items.'
}

In [3]:
#Run business understanding graph
#Explore dataset agentically
state = AgentState(
    business_profile=business_profile,
    datasets={},
    new_datasets=datasets,
    schema_context="",
    explored_datasets=[],
    memory_log=[]
)

understand_graph = build_understand_graph()
result = understand_graph.invoke(state)

state = AgentState(**result)

## Build BD Agent

In [4]:
from agents.bd_agent.graph.state import BDState
from agents.bd_agent.graph.nodes import get_bd_agent
from langchain_core.messages import HumanMessage

user_prompt = "What are the trendy menu I should add to my cafe's"
# user_prompt = "How do sales in each day of the week compared?"
bd_state = BDState(
    messages=[HumanMessage(content=f"<question>{user_prompt}</question>")],
    schema_context=state.schema_context,
    business_profile = state.business_profile,
    datasets = state.datasets
)

agent_node = get_bd_agent()
result = agent_node.invoke(bd_state)


2025-04-27 10:16:31,794 - stratpilot - INFO - [Search and Summary]: [HumanMessage(content="<question>What are the trendy menu I should add to my cafe's</question>", additional_kwargs={}, response_metadata={}, id='726cd02f-1782-480a-b067-398a324066a9'), AIMessage(content='To provide a thoughtful recommendation on trendy menu items to add to your cafe, I need to understand the current trends in the cafe and specialty coffee market, especially those appealing to young professionals and students in urban areas. This will help align the menu additions with your target customer base and competitive environment.\n\nI will perform a web search with up to 2 queries to gather insights on:\n1. Current trendy cafe menu items for specialty coffee shops.\n2. Popular seasonal and Instagram-friendly food and beverage items in cafes.\n\nThis will help me recommend menu items that can increase repeat customers and promote seasonal offerings effectively.\n\nProceeding with the web search now.', additiona

In [5]:
result

{'messages': [HumanMessage(content="<question>What are the trendy menu I should add to my cafe's</question>", additional_kwargs={}, response_metadata={}, id='726cd02f-1782-480a-b067-398a324066a9'),
  AIMessage(content='To provide a thoughtful recommendation on trendy menu items to add to your cafe, I need to understand the current trends in the cafe and specialty coffee market, especially those appealing to young professionals and students in urban areas. This will help align the menu additions with your target customer base and competitive environment.\n\nI will perform a web search with up to 2 queries to gather insights on:\n1. Current trendy cafe menu items for specialty coffee shops.\n2. Popular seasonal and Instagram-friendly food and beverage items in cafes.\n\nThis will help me recommend menu items that can increase repeat customers and promote seasonal offerings effectively.\n\nProceeding with the web search now.', additional_kwargs={'tool_calls': [{'id': 'call_QUYEkGx1aUXb2bC